In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests

In [2]:
# API key and URL
api_key = 'Bearer b19_eQfzPJHYB3SiBBxLYnn13NA8BzNHYe8AF6F0j90IIZ5EnqnHD3XBbPFv14iUQyLy_1L8A74TLDPIMjT4it-s63GM9BNZF0Hl6iulzIyx-piOIKAJvDTvon5uZXYx'
api_url = 'https://api.yelp.com/v3/businesses/search'

# Params
params = {
    'term': 'grocery',
    'location': 'Austin',
    'categories': 'grocery',
    'limit': 50,
    'offset': 0
}

# Set up headers
headers = {
    'Authorization': api_key
}

# Empty list for results
all_groceries = []
results_per_request = 50

while True:
    # API request
    response = requests.get(api_url, params=params, headers=headers)

    # If request successful
    if response.status_code == 200:
        data = response.json()
        businesses = data.get('businesses', [])
        all_groceries.extend(businesses)

        # Check if more results
        if len(businesses) < results_per_request:
            break

        # Update offset for the next request
        params['offset'] += results_per_request

    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        break

# Create list of dictionaries
grocery_data = []
for business in all_groceries:
    coordinates = business.get('coordinates', {})
    grocery_data.append({
        'Name': business.get('name', ''),
        'Address': ', '.join(business.get('location', {}).get('display_address', [])),
        'Ratings': business.get('rating', 0),
        'Latitude': coordinates.get('latitude', 0),
        'Longitude': coordinates.get('longitude', 0)
    })

# Create a Pandas DataFrame
grocery_df = pd.DataFrame(grocery_data)
grocery_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       346 non-null    object 
 1   Address    346 non-null    object 
 2   Ratings    346 non-null    float64
 3   Latitude   346 non-null    float64
 4   Longitude  346 non-null    float64
dtypes: float64(3), object(2)
memory usage: 13.6+ KB


In [3]:
# Create DF
groceries_df = grocery_df[['Name', 'Address', 'Ratings', 'Latitude', 'Longitude']].copy()

# Rename columns
groceries_df.columns = ['grocery_name', 'grocery_address', 'grocery_ratings', 'latitude', 'longitude']

# Add new column for grocery_id
groceries_df['grocery_id'] = range(1, len(groceries_df) + 1)

# Reorder columns
groceries_df = groceries_df[['grocery_id', 'grocery_name', 'grocery_address', 'grocery_ratings', 'latitude', 'longitude']]
print(groceries_df.head())

   grocery_id                  grocery_name  \
0           1                         H-E-B   
1           2  Central Market - North Lamar   
2           3                 ATX Food Mart   
3           4        Sprouts Farmers Market   
4           5             Lake Austin H-E-B   

                                     grocery_address  grocery_ratings  \
0                   1000 E 41st St, Austin, TX 78751              3.5   
1                4001 N Lamar Blvd, Austin, TX 78756              4.0   
2              3002 S Congress Ave, Austin, TX 78704              5.0   
3  1201 Barbara Jordan Blvd, Bldg 7, Austin, TX 7...              5.0   
4            2652 Lake Austin Blvd, Austin, TX 78703              4.0   

    latitude  longitude  
0  30.300643 -97.719963  
1  30.307620 -97.739877  
2  30.232479 -97.758972  
3  30.304818 -97.709573  
4  30.283791 -97.775821  


In [4]:
# Export DF as CSV file
groceries_df.to_csv("Resources/groceries.csv", index=False)


In [5]:
# Load property data from CSV
property_df = pd.read_csv("Resources/property.csv")

In [6]:
from geopy.distance import great_circle

# Junction Table for property_grocery
property_grocery_df = pd.DataFrame(columns=['property_id', 'grocery_id'])

# Find nearby locations and create relationships
for property_row in property_df.itertuples():
    property_location = (property_row.latitude, property_row.longitude)
    
    for grocery_row in groceries_df.itertuples():
        grocery_location = (grocery_row.latitude, grocery_row.longitude)
        
        # Calculate distance between property and grocery locations
        distance_km = great_circle(property_location, grocery_location).kilometers
        
        # Define a distance threshold (adjust as needed)
        distance_threshold = 1.0  
        
        # If the distance is within the threshold, create a relationship
        if distance_km <= distance_threshold:
            new_row = pd.DataFrame({
                'property_id': [property_row.property_id],
                'grocery_id': [grocery_row.grocery_id]
            })
            property_grocery_df = pd.concat([property_grocery_df, new_row], ignore_index=True)

# Display the resulting DataFrame
print(property_grocery_df)


     property_id grocery_id
0              9        137
1              9        238
2             10        137
3             10        238
4             33         99
...          ...        ...
7043        8877        270
7044        8878        240
7045        8878        270
7046        8879        270
7047        8880        270

[7048 rows x 2 columns]


In [7]:
# Export junction table as CSV file
property_grocery_df.to_csv("Resources/property_grocery.csv", index=False)